In [1]:
import sys
#sys.path.insert(0, '/content/drive/MyDrive/pg/siamese_triplet_net/src/')
#sys.path.insert(0,'C:/Users/Mey/Documents/pg-coffee-main/siamese_triplet_net/src/')
sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
import torch
device = torch.cuda.is_available()
from dataloaders import get_train_transforms, get_val_transforms, get_siamese_dataloader, get_triplet_dataloader
from networks import SiameseNet, TripletNet 
from models import *
from losses import ContrastiveLoss, TripletLoss
from trainer import fit
import torchvision
#from CustomViT import CustomViT
# model & optimizer & lr_scheduler

#=== دو خط زیر موقتا کامنت شده است آنرا برگردانیم
embedding_net = ViTEmbeddingNet()
model = TripletNet(embedding_net=embedding_net)

#embedding_net = ViTEmbeddingNet()
#model = TripletNet(embedding_net=embedding_net)

#model = CustomViT(num_classes=15)

#====این قسمت به صورت موقت برای لود امبدینگ مدل آموزش داده شده قبلی بر روی پلنت ویلج به کار میرود
#import torch
#model = torch.load("C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_NFNet_15Shot.h5",map_location=torch.device('cpu'))
#model.load_state_dict(torch.load("C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_NFNet_15Shot.h5",map_location=torch.device('cpu')) )
#Wmodel.eval()
#============================
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss_fn = TripletLoss(1.)
n_epochs=7 #100

if device:
    model.cuda()

log_interval =10 # 100

# path to data
#path_data = '/content/drive/MyDrive/pg/dataset/'
path_data='f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'

# define siamese train and val loaders
# this loader is implemented for datasets in ImageFolder format (https://pytorch.org/vision/stable/datasets.html#imagefolder)
triplet_train_loader = get_triplet_dataloader(root=path_data + '/train/', batch_size=5, transforms=get_train_transforms())
triplet_val_loader = get_triplet_dataloader(root=path_data + '/val/', batch_size=5, transforms=get_val_transforms())
fit(triplet_train_loader, triplet_val_loader, model, loss_fn, optimizer, lr_scheduler, n_epochs, device, log_interval)


train_data = torchvision.datasets.ImageFolder(root=path_data + '/train/', transform=get_val_transforms())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32)

test_data = torchvision.datasets.ImageFolder(root=path_data + '/test/', transform=get_val_transforms())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

x_train, y_train = generate_embeddings(train_loader, model)
x_test, y_true = generate_embeddings(test_loader, model)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

classifier = KNeighborsClassifier(n_neighbors=1)
# classifier = SVC()
# classifier = SGDClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

accuracy = round(accuracy_score(y_true, y_pred)*100, 2)
precision = round(precision_score(y_true, y_pred, average='macro')*100, 2)
recall = round(recall_score(y_true, y_pred, average='macro')*100, 2)
f1 = round(f1_score(y_true, y_pred, average='macro')*100, 2)
print(f'--- Results for MobileNetv2 Embeddings on KNN (k = 1) ---')
print(f'Accuracy Score:{accuracy}')
print(f'Precision Score: {precision}')
print(f'Recall Score: {recall}')
print(f'F1 Score: {f1}')

C:\Users\Mey\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Mey\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\Mey\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\Mey\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resum

KeyboardInterrupt: 